In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import matplotlib.pyplot as plt
import numpy as np

# Dictionary of pipe materials and C-factors
pipe_materials = {
    "Copper (new)": 150,
    "PVC": 150,
    "Galvanized Steel": 120,
    "Cast Iron": 100,
    "Asbestos Cement": 140,
    "Concrete (old)": 80,
    "Other (Custom)": None
}

# Unit conversion functions
def mm_to_in(mm): return mm / 25.4
def m_to_ft(m): return m * 3.28084

# Hazen-Williams calculation
def hazen_williams_pressure_drop(length_ft, flow_gpm, diameter_in, c_factor):
    head_loss = 10.67 * (length_ft * flow_gpm**1.85) / (c_factor**1.85 * diameter_in**4.87)
    pressure_drop = 0.433 * head_loss
    return head_loss, pressure_drop

# Widgets
unit_selector = widgets.ToggleButtons(
    options=['Imperial', 'Metric'],
    description='Units:',
    style={'button_color': 'lightgray'}
)

material_selector = widgets.Dropdown(
    options=pipe_materials.keys(),
    value="Copper (new)",
    description="Material:"
)

custom_c_input = widgets.FloatText(
    value=120.0, description="Custom C:", disabled=True
)

length_input = widgets.FloatText(value=100.0, description="Length:")
flow_input = widgets.FloatText(value=100.0, description="Flow (gpm):")
diameter_input = widgets.FloatText(value=4.0, description="Diameter:")

show_graph = widgets.Checkbox(value=False, description="Show Pressure Drop Chart")
calc_button = widgets.Button(description="Calculate", button_style="success")
output = widgets.Output()

# Update C-factor input based on material
def on_material_change(change):
    selected = change['new']
    c_val = pipe_materials[selected]
    custom_c_input.disabled = c_val is not None
    if c_val is not None:
        custom_c_input.value = c_val

material_selector.observe(on_material_change, names='value')

# Update labels for metric units
def on_unit_change(change):
    if change['new'] == 'Metric':
        length_input.description = "Length (m):"
        diameter_input.description = "Diameter (mm):"
    else:
        length_input.description = "Length (ft):"
        diameter_input.description = "Diameter (in):"

unit_selector.observe(on_unit_change, names='value')

# Calculation logic
def on_calc_clicked(b):
    with output:
        clear_output()

        units = unit_selector.value
        length = length_input.value
        flow = flow_input.value
        diameter = diameter_input.value
        c_factor = custom_c_input.value

        # Convert units if metric
        if units == 'Metric':
            length = m_to_ft(length)
            diameter = mm_to_in(diameter)

        # Calculate
        head_loss, pressure_drop = hazen_williams_pressure_drop(length, flow, diameter, c_factor)

        # Display results
        display(Markdown("### 📊 Results"))
        display(Markdown(f"- **Head Loss**: `{head_loss:.2f} ft`"))
        display(Markdown(f"- **Pressure Drop**: `{pressure_drop:.2f} psi`"))

        # Optional graph
        if show_graph.value:
            lengths = np.linspace(0, length, 100)
            head_losses = 10.67 * (lengths * flow**1.85) / (c_factor**1.85 * diameter**4.87)
            pressures = 0.433 * head_losses

            plt.figure(figsize=(6, 4))
            plt.plot(lengths, pressures, linewidth=2)
            plt.title("Pressure Drop vs Pipe Length")
            plt.xlabel("Length (ft)")
            plt.ylabel("Pressure Drop (psi)")
            plt.grid(True)
            plt.tight_layout()
            plt.show()

# Bind calculation
calc_button.on_click(on_calc_clicked)

# Layout
display(widgets.VBox([
    unit_selector,
    material_selector,
    custom_c_input,
    length_input,
    flow_input,
    diameter_input,
    show_graph,
    calc_button,
    output
]))